In [1]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/tweet-sentiment-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1735471095'}]}


In [4]:
import json
import requests
import tensorflow as tf
from pprint import PrettyPrinter
import base64

def create_tf_example(text, id_num=1):
    """
    Create a TensorFlow Example message with all required features.
    
    Args:
        text (str): The tweet text to classify
        id_num (int): An ID number for the example
    """
    feature = {
        'tweet': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])
        ),
        'id': tf.train.Feature(
            int64_list=tf.train.Int64List(value=[id_num])
        )
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature)
    )
    
    return example.SerializeToString()

def make_prediction(text, server_url='http://localhost:8080'):
    """
    Make predictions using deployed TensorFlow Serving model
    
    Args:
        text (str): Input text to classify
        server_url (str): URL where TensorFlow Serving is running
    
    Returns:
        float: Prediction probability (0-1)
    """
    # Construct the REST API endpoint
    model_name = 'tweet-sentiment-model'
    predict_url = f"{server_url}/v1/models/{model_name}:predict"
    
    # Create TF Example
    tf_example = create_tf_example(text)
    
    # Create the request body
    input_data = {
    "instances": [
        {"b64": base64.b64encode(tf_example).decode('utf-8')}
    ]
}

    
    # Debug information
    print(f"\nMaking request to: {predict_url}")
    print(f"Input text: {text}")
    
    try:
        # First verify the model is available
        model_status_url = f"{server_url}/v1/models/{model_name}"
        status_response = requests.get(model_status_url)
        status_response.raise_for_status()
        print("\nModel status:")
        pp = PrettyPrinter()
        pp.pprint(status_response.json())
        
        # Make prediction request
        headers = {"Content-Type": "application/json"}
        response = requests.post(predict_url, json=input_data, headers=headers)
        
        # Print raw response for debugging
        print("\nRaw response:")
        print(response.text)
        
        response.raise_for_status()
        
        # Parse the response
        result = response.json()
        print("\nParsed response:")
        pp.pprint(result)
        
        prediction = result['predictions'][0][0]  # Get first prediction
        return float(prediction)
        
    except requests.exceptions.RequestException as e:
        print(f"\nError details:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response status code: {e.response.status_code}")
            print(f"Response text: {e.response.text}")
        return None

# Example usage
if __name__ == "__main__":
    # Test texts
    texts = [
        "I love this product! It's amazing!",
        "This is terrible, I hate it.",
    ]
    
    print("\nTesting prediction service...")
    for i, text in enumerate(texts, 1):
        print(f"\nTesting text {i}: {text}")
        prediction = make_prediction(text)
        
        if prediction is not None:
            sentiment = "negative" if prediction >= 0.5 else "positive"
            print(f"Prediction score: {prediction:.3f}")
            print(f"Sentiment: {sentiment}")


Testing prediction service...

Testing text 1: I love this product! It's amazing!

Making request to: http://localhost:8080/v1/models/tweet-sentiment-model:predict
Input text: I love this product! It's amazing!

Model status:
{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1735471095'}]}

Raw response:
{
    "predictions": [[0.0193230305]
    ]
}

Parsed response:
{'predictions': [[0.0193230305]]}
Prediction score: 0.019
Sentiment: positive

Testing text 2: This is terrible, I hate it.

Making request to: http://localhost:8080/v1/models/tweet-sentiment-model:predict
Input text: This is terrible, I hate it.

Model status:
{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1735471095'}]}

Raw response:
{
    "predictions": [[0.748471618]
    ]
}

Parsed 